## パーセプトロン
- ニューラルネットワークの起源と書いてある
- 複数入力を2値の1出力で返す変換器、論理素子モデル

それぞれの入力値に対して対応する重みをかけた計算結果が、  
閾値hを超えていれば1、超えなければ0を出力する。

画像はhttp://nkdkccmbr.hateblo.jp/entry/2016/10/06/222245
を使わせて頂くとするmm

図ch001-1:単純パーセプトロン
<img border="0" src="./images/perceptron.png" width="512" height="512" alt="イラスト1">

xi(i=1..k): 入力信号  
wi(i=1..k): 重み  
y: 出力信号  

重みwは電気回路の電流で言う所のコンダクタンス(1/抵抗)的なもので、重みが大きいほど発火しやすい。

In [1]:
# パーセプトロンの式(k=2)：
def perceptron(x1, x2, w1, w2, h):
    return step_func(x1*w1 + x2*w2, h)

# ステップ関数
def step_func(value, threshold):
    if value > threshold:
        return 1
    else:
        return 0

# AND回路をパーセプトロンで作る、w1、w2、hは学習した値ではなくAND論理になるように人間が考えた値
AND = (lambda x1, x2: perceptron(x1, x2, 0.5, 0.5, 0.7))
print(AND(0, 0))
print(AND(1, 1))
print(AND(1, 0))
print(AND(0, 1))

0
1
0
0


In [2]:
# NAND論理
NAND = (lambda x1, x2: perceptron(x1, x2, -0.5, -0.5, -0.7))
print(NAND(0, 0))
print(NAND(1, 1))
print(NAND(1, 0))
print(NAND(0, 1))

1
0
1
1


NAND論理が実現できるということは、パーセプトロンを多層で組み合わせれば所望の論理回路を実現することが出来る。   
ASICの一種のセルベースICはあらかじめNANDだけを敷き詰めておいて後から配線をすることでセミカスタムにLSIを設計する。  

例えば、NOT演算を行いたい場合には二つの入力を同じものにすれば実現できる。  
ド・モルガンの法則などでゴニョゴニョすればどのような論理もNANDだけで組む事ができる。  

$$
\begin{align}
\overline{A \cdot A} = \overline{A}
\end{align}
$$

XORをNANDで実現する(空集合と分配則を使うと4素子に出来る)

$$
\begin{align}
A \cdot \overline{B} + \overline{A} \cdot B
= \overline{\overline{A \cdot \overline{B} + \overline{A} \cdot B}}
= \overline{ \overline{A \cdot \overline{B}} \cdot \overline{\overline{A} \cdot B} }
= \overline{ \overline{A \cdot \overline{B \cdot B}} \cdot \overline{\overline{A \cdot A} \cdot B} }
\end{align}
$$

In [3]:
# 上の式をコードに落としてみる
layer1_NOTA = (lambda x1: NAND(x1, x1))
layer1_NOTB = (lambda x2: NAND(x2, x2))
layer2_A_NOTB = (lambda x1, x2: NAND(x1, layer1_NOTB(x2)))
layer2_B_NOTA = (lambda x1, x2: NAND(x2, layer1_NOTA(x1)))

NOR = (lambda x1, x2: NAND(layer2_A_NOTB(x1, x2), layer2_B_NOTA(x1, x2)))
print(NOR(0, 0))
print(NOR(1, 1))
print(NOR(1, 0))
print(NOR(0, 1))

0
0
1
1


### 単純パーセプトロンの限界
式を見て明らかなように単純パーセプトロンは線形分離しかできない。  
排他的論理和XORのような非線形の場合は分離ができない。  
しかし、上述したように多層パーセプトロンは非線形分離も出来る。